In [28]:
#load csv file into a pandas dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

conn = sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db')
c = conn.cursor()

## Add additional information to database

#### Add subjects xlsx

In [ ]:
quantex_subjects_df = pd.read_excel('/home/nele_pauline_suffo/ProcessedData/quantex_subjects.xlsx')
quantex_subjects_df

# add dataframe as new table into the database
quantex_subjects_df.to_sql('Subjects', conn, if_exists='replace', index=False)
# convert birthday column to DATE format
c.execute('''
UPDATE Subjects
SET birthday = DATE(birthday)
''')
conn.commit()

### Add age group to age_mapping table 

In [ ]:

#load the data
df = pd.read_excel('/home/nele_pauline_suffo/ProcessedData/quantex_at_home_data_sheet.xlsx')

df_age_group = df[["file name  (generated automatically)", "Age Group"]]
# remove the rows with missing values
df_age_group = df_age_group.dropna()

# Add .MP4 extension to file names
df_age_group["file name  (generated automatically)"] = df_age_group["file name  (generated automatically)"].astype(str) + ".MP4"

#store those two columns in a dictionary
age_group_dict = df_age_group.set_index("file name  (generated automatically)").to_dict()
age_group_dict = age_group_dict["Age Group"]

# Create table for age group mapping
cursor.execute("""
CREATE TABLE age_mapping (
    video_path TEXT,
    age_group INTEGER
)""")

# Insert age group mapping data into table
cursor.executemany("INSERT INTO age_mapping (video_path, age_group) VALUES (?, ?)", age_group_dict.items())
conn.commit()

# Add new column (SQLite doesn't support ALTER TABLE ADD COLUMN IF NOT EXISTS)
try:
    cursor.execute("ALTER TABLE VideoStatistics ADD COLUMN age_group INTEGER")
    conn.commit()
except sqlite3.OperationalError:
    print("Column already exists, continuing...")

In [33]:
# Update videostatistics using the age_mapping table
update_query = """
UPDATE VideoStatistics 
SET age_group = (
    SELECT am.age_group
    FROM Videos v
    JOIN age_mapping am ON v.video_path = am.video_path
    WHERE v.video_id = VideoStatistics.video_id
)
"""
cursor.execute(update_query)
conn.commit()

## Plotting

## Statistics

In [22]:
# query sum of number of frames from all videos in database
c.execute('SELECT SUM(total_frames) FROM VideoStatistics')
total_frames = c.fetchone()[0]
total_frames

19023571

In [ ]:
# first add id as new column in Videos table generated from the video_path
c.execute('''
ALTER TABLE Videos
ADD COLUMN id INTEGER
ADD COLUMN recording_date DATE
''')
conn.commit()

## Update both columns with properly formatted data
c.execute('''
UPDATE Videos
SET 
    id = CAST(
        SUBSTR(
            video_path,
            INSTR(video_path, 'id') + 2,
            INSTR(SUBSTR(video_path, INSTR(video_path, 'id') + 2), '_')
        ) AS INTEGER
    ),
    recording_date = REPLACE(
        SUBSTR(
            video_path,
            INSTR(video_path, '_20') + 1,  -- finds the position of '_20' and adds 1
            10  -- length of YYYY_MM_DD
        ),
        '_', '-'
    )
''')
conn.commit()

In [51]:
# query from Videos table all video_paths, extract from the path the recording date and id, then use the id to get birthday from Subjects table
c.execute('''
SELECT 
    v.id,
    v.video_id,
    v.video_path,
    date(s.birthday) as birthday,
    date(v.recording_date) as recording_date
FROM Videos v
JOIN Subjects s ON v.id = s.id
ORDER BY v.id, v.recording_date
''')

# Convert to DataFrame and ensure proper date types
video_birthdays = pd.DataFrame(c.fetchall(), columns=['id', 'video_id', 'video_path', 'birthday', 'recording_date'])
video_birthdays['birthday'] = pd.to_datetime(video_birthdays['birthday'])
video_birthdays['recording_date'] = pd.to_datetime(video_birthdays['recording_date'])

# Calculate age
video_birthdays['age'] = ((video_birthdays['recording_date'] - video_birthdays['birthday']).dt.days / 365.25).round(2)

# Display results
video_birthdays

,id,video_id,video_path,birthday,recording_date,age
0,254922,345,quantex_at_home_id254922_2022_04_12_01.MP4,2016-05-28,2022-04-12,5.87
1,254922,320,quantex_at_home_id254922_2022_04_19_01.MP4,2016-05-28,2022-04-19,5.89
2,254922,365,quantex_at_home_id254922_2022_04_21_01.MP4,2016-05-28,2022-04-21,5.90
3,254922,394,quantex_at_home_id254922_2022_05_21_02.MP4,2016-05-28,2022-05-21,5.98
4,254922,439,quantex_at_home_id254922_2022_05_21_01.MP4,2016-05-28,2022-05-21,5.98
...,...,...,...,...,...,...
448,275411,281,quantex_at_home_id275411_2024_11_22_03.MP4,2020-10-21,2024-11-22,4.09
449,275411,187,quantex_at_home_id275411_2024_11_23_03.MP4,2020-10-21,2024-11-23,4.09
450,275411,227,quantex_at_home_id275411_2024_11_23_02.MP4,2020-10-21,2024-11-23,4.09
451,275411,233,quantex_at_home_id275411_2024_11_23_04.MP4,2020-10-21,2024-11-23,4.09


In [16]:
frames_with_detections_query = """
WITH FramesWithDetections AS (
    -- First get unique frames with detections for each class and video
    SELECT 
        v.id,
        v.video_id,
        d.frame_number,
        MAX(CASE WHEN d.object_class = 1.0 THEN 1 ELSE 0 END) as has_adult,
        MAX(CASE WHEN d.object_class = 0.0 THEN 1 ELSE 0 END) as has_child,
        MAX(CASE WHEN d.object_class = 3.0 THEN 1 ELSE 0 END) as has_adult_face,
        MAX(CASE WHEN d.object_class = 2.0 THEN 1 ELSE 0 END) as has_child_face,
        MAX(CASE WHEN d.object_class = 5.0 THEN 1 ELSE 0 END) as has_book,
        MAX(CASE WHEN d.object_class = 6.0 THEN 1 ELSE 0 END) as has_toy,
        MAX(CASE WHEN d.object_class = 7.0 THEN 1 ELSE 0 END) as has_kitchenware,
        MAX(CASE WHEN d.object_class = 8.0 THEN 1 ELSE 0 END) as has_screen,
        MAX(CASE WHEN d.object_class = 9.0 THEN 1 ELSE 0 END) as has_food,
        MAX(CASE WHEN d.object_class = 10.0 THEN 1 ELSE 0 END) as has_other_object
    FROM Videos v
    JOIN Detections d ON v.video_id = d.video_id
    GROUP BY v.id, v.video_id, d.frame_number
),
TotalFramesPerChild AS (
    -- Calculate total frames per child ID
    SELECT 
        v.id,
        SUM(vs.total_frames) as total_frames
    FROM Videos v
    JOIN VideoStatistics vs ON v.video_id = vs.video_id
    GROUP BY v.id
)
-- Then aggregate detections by child ID
SELECT 
    f.id,
    COUNT(DISTINCT f.video_id) as video_count,
    t.total_frames,
    SUM(f.has_adult) as frames_with_adult,
    SUM(f.has_child) as frames_with_child,
    SUM(f.has_adult_face) as frames_with_adult_face,
    SUM(f.has_child_face) as frames_with_child_face,
    SUM(f.has_book) as frames_with_book,
    SUM(f.has_toy) as frames_with_toy,
    SUM(f.has_kitchenware) as frames_with_kitchenware,
    SUM(f.has_screen) as frames_with_screen,
    SUM(f.has_food) as frames_with_food,
    SUM(f.has_other_object) as frames_with_other_object
FROM FramesWithDetections f
JOIN TotalFramesPerChild t ON f.id = t.id
JOIN VideoStatistics vs ON f.video_id = vs.video_id
GROUP BY f.id, t.total_frames
ORDER BY f.id;
"""

# Execute query and create DataFrame
frames_with_detections = pd.read_sql(frames_with_detections_query, conn)


# Calculate percentages
for col in frames_with_detections.columns[3:]:  # Skip id, video_count, and total_frames
    frames_with_detections[f'{col}_percent'] = (
        frames_with_detections[col] / frames_with_detections['total_frames'] * 100
    ).round(2)

# Display results
display(frames_with_detections)

,id,video_count,total_frames,frames_with_adult,frames_with_child,frames_with_adult_face,frames_with_child_face,frames_with_book,frames_with_toy,frames_with_kitchenware,...,frames_with_adult_percent,frames_with_child_percent,frames_with_adult_face_percent,frames_with_child_face_percent,frames_with_book_percent,frames_with_toy_percent,frames_with_kitchenware_percent,frames_with_screen_percent,frames_with_food_percent,frames_with_other_object_percent
0,254922,8,363585,6731,4320,4534,1753,690,1508,106,...,1.85,1.19,1.25,0.48,0.19,0.41,0.03,0.00,0.00,0.46
1,255237,6,282993,5405,15208,1570,6359,1165,7373,50,...,1.91,5.37,0.55,2.25,0.41,2.61,0.02,0.02,0.12,1.05
2,255695,4,68763,2165,2357,780,1008,5,881,630,...,3.15,3.43,1.13,1.47,0.01,1.28,0.92,0.00,0.00,0.47
3,255706,2,72922,3693,2122,1178,1359,379,1557,458,...,5.06,2.91,1.62,1.86,0.52,2.14,0.63,0.00,0.00,0.25
4,255944,5,138006,3224,4767,849,2661,2067,1244,244,...,2.34,3.45,0.62,1.93,1.50,0.90,0.18,0.00,0.03,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,271611,7,286457,2972,2895,2210,232,1005,3994,471,...,1.04,1.01,0.77,0.08,0.35,1.39,0.16,0.01,0.06,0.96
70,271700,6,200637,9361,2115,5093,1233,736,1244,130,...,4.67,1.05,2.54,0.61,0.37,0.62,0.06,0.02,0.04,1.06
71,272475,4,171070,5154,1768,1888,868,1211,676,169,...,3.01,1.03,1.10,0.51,0.71,0.40,0.10,0.01,0.02,1.17
72,273855,6,303755,6960,1994,2302,656,588,1060,74,...,2.29,0.66,0.76,0.22,0.19,0.35,0.02,0.01,0.01,0.47


In [49]:
# save frames_with_detections and video_birthdays to the same csv file
frames_with_detections.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/frames_with_detections.csv', index=False)
video_birthdays.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/video_birthdays.csv', index=False)

## Plotting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests

# First, get raw data with proper summed normalization
raw_data_query = """
SELECT 
    age_group,
    video_id,
    CAST(child_count AS FLOAT) / total_frames as norm_child_count,
    CAST(adult_count AS FLOAT) / total_frames as norm_adult_count,
    CAST(child_face_count AS FLOAT) / total_frames as norm_child_face_count,
    CAST(adult_face_count AS FLOAT) / total_frames as norm_adult_face_count,
    CAST(toy_count AS FLOAT) / total_frames as norm_toy_count,
    CAST(book_count AS FLOAT) / total_frames as norm_book_count,
    CAST(kitchenware_count AS FLOAT) / total_frames as norm_kitchenware_count,
    CAST(screen_count AS FLOAT) / total_frames as norm_screen_count,
    CAST(other_object_count AS FLOAT) / total_frames as norm_other_object_count
FROM VideoStatistics
WHERE age_group IS NOT NULL;
"""

raw_data = pd.read_sql(raw_data_query, conn)

# Define classes and display names
classes = ['norm_child_count', 'norm_adult_count', 'norm_child_face_count',
           'norm_adult_face_count', 'norm_toy_count', 'norm_book_count',
           'norm_kitchenware_count', 'norm_screen_count', 'norm_other_object_count']

class_display_names = {
    'norm_child_count': 'Child',
    'norm_adult_count': 'Adult',
    'norm_child_face_count': 'Child Face',
    'norm_adult_face_count': 'Adult Face',
    'norm_toy_count': 'Toy',
    'norm_book_count': 'Book',
    'norm_kitchenware_count': 'Kitchenware',
    'norm_screen_count': 'Screen',
    'norm_other_object_count': 'Other Objects'
}

# Perform Kruskal-Wallis tests and track significant classes
test_results = []
significant_classes = []

for class_name in classes:
    groups = [group[class_name].values 
             for name, group in raw_data.groupby('age_group')]
    h_stat, p_val = stats.kruskal(*groups)
    
    test_results.append({
        'class_name': class_name,
        'display_name': class_display_names[class_name],
        'h_statistic': h_stat,
        'p_value': p_val
    })
    
    if p_val < 0.05:
        significant_classes.append(class_name)

significance_results = pd.DataFrame(test_results)

# Perform pairwise tests only for significant classes with Bonferroni correction
def perform_pairwise_tests(data, class_name, display_name):
    age_groups = sorted(data['age_group'].unique())
    results = []
    
    for i in range(len(age_groups)):
        for j in range(i+1, len(age_groups)):  
            group1 = data[data['age_group'] == age_groups[i]][class_name]
            group2 = data[data['age_group'] == age_groups[j]][class_name]
            
            stat, pval = stats.mannwhitneyu(group1, group2, alternative='two-sided')
            
            results.append({
                'Class': display_name,
                'Group 1': age_groups[i],
                'Group 2': age_groups[j],
                'Statistic': stat,
                'p-value': pval
            })
    
    return results

# Collect and correct pairwise results
all_pairwise_results = []
for class_name in significant_classes:
    display_name = class_display_names[class_name]
    results = perform_pairwise_tests(raw_data, class_name, display_name)
    all_pairwise_results.extend(results)

pairwise_results_df = pd.DataFrame(all_pairwise_results)

# Apply Bonferroni correction
pairwise_results_df['p_adjusted'] = multipletests(
    pairwise_results_df['p-value'], method='bonferroni'
)[1]

# Add significance labels
pairwise_results_df['Significance'] = np.where(
    pairwise_results_df['p_adjusted'] < 0.001, '***',
    np.where(pairwise_results_df['p_adjusted'] < 0.01, '**',
    np.where(pairwise_results_df['p_adjusted'] < 0.05, '*', 'ns'))
)

# Prepare melted data for plotting
normalized_results_melted = pd.melt(
    raw_data,
    id_vars=['age_group'],
    value_vars=[c for c in classes],
    var_name='class',
    value_name='normalized_count'
)
normalized_results_melted['display_name'] = normalized_results_melted['class'].map(class_display_names)

# Create annotation dictionary from significant pairwise results
sig_pairwise = pairwise_results_df[pairwise_results_df['Significance'] != 'ns']
annotation_dict = {}
for _, row in sig_pairwise.iterrows():
    key = (row['Class'], row['Group 1'], row['Group 2'])
    annotation_dict[key] = row['Significance']

# Define custom colors for age groups
age_group_colors = {
    3: '#8D8E3D',  # leuphana green
    4: '#691633',  # leuphana red
    5: '#B3B6B0',  # leuphana blue
}
# Create visualization with significance bars
plt.figure(figsize=(18, 10))
ax = sns.barplot(
    x='display_name',
    y='normalized_count',
    hue='age_group',
    data=normalized_results_melted,
    order=class_display_names.values(),
    palette=age_group_colors
)

# Set fixed y-axis limit to 0.04
y_max_fixed = 0.055
ax.set_ylim(0, y_max_fixed)

# Define class-specific base heights
class_base_heights = {
    'Child': 0.045,         
    'Adult': 0.04,         
    'Child Face': 0.012,   
    'Adult Face': 0.04,    
    'Toy': 0.013,          
    'Book': 0.008,         
    'Kitchenware': 0.003,  
    'Screen': 0.04,      
    'Other Objects': 0.04
}

# Define spacing parameters
bracket_step = 0.0015  # Space between stacked brackets

# Track current height for each class to avoid overlaps
class_heights = {class_name: base_height for class_name, base_height in class_base_heights.items()}

# Get x positions for each category
x_positions = np.arange(len(class_display_names))
bar_width = 0.8 / len(raw_data['age_group'].unique())

# Loop through significant comparisons and draw brackets
for i, class_name in enumerate(class_display_names.values()):
    for j in range(len(age_groups)):
        for k in range(j+1, len(age_groups)):
            group1 = age_groups[j]
            group2 = age_groups[k]
            
            if (class_name, group1, group2) in annotation_dict:
                significance = annotation_dict[(class_name, group1, group2)]
                
                # Calculate x positions for bars
                x_pos1 = x_positions[i] + (j - len(age_groups)/2 + 0.5) * bar_width
                x_pos2 = x_positions[i] + (k - len(age_groups)/2 + 0.5) * bar_width
                
                # Get the current height for this class and increment for next
                y_pos = class_heights[class_name]
                class_heights[class_name] += bracket_step
                
                # Draw the bracket
                ax.plot([x_pos1, x_pos1, x_pos2, x_pos2], 
                        [y_pos, y_pos + bracket_step * 0.3, y_pos + bracket_step * 0.3, y_pos], 
                        color='black', linewidth=1)
                
                # Add the significance label
                ax.text((x_pos1 + x_pos2) / 2, y_pos + bracket_step * 0.2, significance, 
                       ha='center', va='bottom')

total_videos = len(raw_data['video_id'].unique())
total_frames_query = """
SELECT SUM(total_frames) as total_frames
FROM VideoStatistics
WHERE age_group IS NOT NULL;
"""
total_frames = pd.read_sql(total_frames_query, conn).iloc[0]['total_frames']

plt.title(f'Comparison of YOLO11x Object Detection Classes Across Age Groups\n(Normalized Counts per Frame)\n'
          f'Applied on Quantex Dataset: {total_videos:,} Videos, {total_frames:,} Frames')
plt.xlabel('')  # Remove x-axis label
plt.ylabel('Normalized Counts (Objects per Frame)')
plt.xticks(rotation=45)
plt.legend(title='Age Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('/home/nele_pauline_suffo/outputs/detection_pipeline_results/quantex_normalized_counts.png')

plt.show()
# save the figure

# Display filtered results
print("## Significant Kruskal-Wallis Results (p < 0.05) ##")
display(significance_results[significance_results['p_value'] < 0.05]
        [['display_name', 'h_statistic', 'p_value']].round(4))

print("\nRaw Pairwise Comparisons:")
# display full p-values for as 0.0000
pd.options.display.float_format = '{:.4f}'.format
display(pairwise_results_df[['Class', 'Group 1', 'Group 2', 'p-value']])

print("\n## Significant Pairwise Comparisons (Bonferroni-corrected) ##")
display(sig_pairwise[['Class', 'Group 1', 'Group 2', 
                     'Statistic', 'p_adjusted', 'Significance']].round(4))


In [ ]:
# Close connection
conn.close()